In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [2]:
from tra.models import *
from tra.test_runs_processing import *
from rep_portal.api import *
from functools import reduce
from django.contrib.auth.models import User
from tra import utils
from stats.analyzer import Analyzer 


In [3]:
def parse_failmessagetypes(ids_list):
    querysets = []
    for _id in ids_list:
        fail_message_type_group = FailMessageTypeGroup.objects.get(id=_id)
        querysets.append(fail_message_type_group.fail_message_types.all())

    fail_message_types = list(chain(*querysets))
    fail_message_dict = {fm_type.regex: fm_type.name for fm_type in fail_message_types}
    return fail_message_dict

def prepare_filters_for_users_all_subs_regfilters(user):
    def _parse_branches(branches):
        branches_new = []q
        for branch in branches:
            if branch.lower() == "trunk":
                branches_new.append("SBTS00")
            else:
                branches_new.append(f"SBTS{branch}")
        return branches_new

    fields_dict = utils.get_distinct_values_based_on_subscribed_regfilters(user)
    test_sets =      [elem["pk"] for elem in fields_dict["test_set_name"]]
    testline_types = [elem["pk"] for elem in fields_dict["testline_type"]]
    branches =       [elem["pk"] for elem in fields_dict["branch"]]
    branches = _parse_branches(branches)
    results = ["environment issue", "not analyzed"]

    return {
        "test_set": ",".join(test_sets),
        "testline_type": ",".join(testline_types),
        "builds": ",".join(branches),
        "result": ",".join(results)
    }



In [33]:
filters = prepare_filters_for_users_all_subs_regfilters(User.objects.get(username="plinta"))
filters["limit"] = 5000
fail_message_dict = parse_failmessagetypes([1,2,3,4,5,6])

date_start, date_end = "2022-09-01", "2022-10-01" 
analyzer = Analyzer(fail_message_dict, filters)
dataframe = analyzer.get_data_from_rp()

# df, label = analyzer.get_data_filtered_by_date_ranges(date_start, date_end)
# data = analyzer.plot_runs_by_exception_types(df=df)

Starting new HTTPS connection (1): rep-portal.wroclaw.nsn-rdnet.net


In [34]:
dataframe_norm_failmsg = analyzer._get_normalized_exception_data(dataframe)

In [43]:
df_dated, _ = analyzer.get_data_filtered_by_date_ranges(dataframe_norm_failmsg, date_start, date_end)

In [44]:
from io import BytesIO
in_memory_excell = BytesIO()
df_dated.to_excel(in_memory_excell)
in_memory_excell.seek(0,0)
with open("dataframe.xlsx", 'wb') as f:
    f.write(in_memory_excell.read())

2